# Elliptic Curve Digital Signature Algorithm (EC DSA)

# Setup

btclib is needed: let's install/update it and import straight away some of its functions

In [5]:
!pip install --upgrade btclib

from hashlib import sha256 as hf

from btclib.ecc.number_theory import mod_inv
from btclib.ecc.curve import mult
from btclib.ecc.curve import secp256k1 as ec

Requirement already up-to-date: btclib in /Users/ferdi/Git/upstream/btclib (2020.9)


For this exercise we use secp256k1 as elliptic curve and SHA256 as hash function:

In [22]:
print(ec)

Curve
 p   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F
 a   = 0
 b   = 7
 x_G = 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798
 y_G = 483ADA77 26A3C465 5DA4FBFC 0E1108A8 FD17B448 A6855419 9C47D08F FB10D4B8
 n   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE BAAEDCE6 AF48A03B BFD25E8C D0364141
 h = 1


Normally, hf is chosen such that its output size is roughly equal to the size of ec.n, since the overall security of the signature scheme will depend on the smallest of the two; however, the ECDSA standard support all combinations of sizes

In [25]:
print(hf().digest_size)
print(ec.nsize)

32
32


# **Digital Signature Protocol**

## 1. Key generation

Private key (generated elsewhere, a fixed value here):

In [8]:
q = 0x18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725
assert 0 < q < ec.n, "Invalid private key"
print("q:", q)
print("Hex(q):", hex(q))

q: 11253563012059685825953619222107823549092147699031672238385790369351542642469
Hex(q): 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725


and the corresponding Public Key:

In [9]:
Q = mult(q)
print(Q)
print("PubKey:", "02" if (Q[1] % 2 == 0) else "03", hex(Q[0]))

(36422191471907241029883925342251831624200921388586025344128047678873736520530, 20277110887056303803699431755396003735040374760118964734768299847012543114150)
PubKey: 02 0x50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352


# 2. Sign Message

## Message
The message to be signed msg is first processed by hf, yielding to the so-called 'non-interactive challenge' c:

In [10]:
msg = "Paolo is afraid of ephemeral random numbers"

# challenge is an integer modulo ec.n:
msghd = hf(msg.encode()).digest()
c = int.from_bytes(msghd, 'big') % ec.n
assert c != 0
print("c:", hex(c))

c: 0x9788fd27b3aafd1bd1591a1158ce2d8bdc37ab4040dddb64e64d17616e69ce2b


## Deterministic Ephemeral Key
An ephemeral key k is required for signing; it must be kept secret and never reused. A good choice is to use a deterministic key: 

`k = hf(q||c) mod n` 

different for each msg, private because of q

In [11]:
k_bytes = hf(q.to_bytes(32, 'big') + c.to_bytes(32, 'big')).digest()
k = int.from_bytes(k_bytes, 'big') % ec.n
assert 0 < k < ec.n, "Invalid ephemeral key"
print("k:", hex(k))

k: 0x3fa3ccf6c168482533f1fa066650704546e56f0bf15fbfb3b4bc51f404e19ee7


## Signature Algorithm

In [12]:
K = mult(k)

r = K[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different k
assert r != 0

s = mod_inv(k, ec.n) * (c + r*q) % ec.n
# if s == 0 (extremely unlikely for large ec.n) go back to a different k
assert s != 0

print("r:", hex(r))
print("s:", hex(s))

r: 0xf91a63e7574b8a7ea99cc8999456e8044b9d1cb05a3ec25c3e8886cee3ed0142
s: 0xb7297f54edd8f8b200b6115bf8ce54336b5ae4c0fe17cb509f11fd09e6751176


# 3. Verify Signature

In [13]:
w = mod_inv(s, ec.n)
u = c*w % ec.n
v = r*w % ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x % ec.n)

True


# Malleated Signature

In [14]:
sm = ec.n - s
print("r :", hex(r))
print("sm:", hex(sm))

r : 0xf91a63e7574b8a7ea99cc8999456e8044b9d1cb05a3ec25c3e8886cee3ed0142
sm: 0x48d680ab1227074dff49eea40731abcb4f53f825b130d4eb20c06182e9c12fcb


Malleated signature verification:

In [15]:
w = mod_inv(sm, ec.n)
u = c*w % ec.n
v = r*w % ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x % ec.n)

True


# A Humongous Mistake

## Second Message
A second different message to be signed and its corresponding challenge:

In [18]:
msg2 = "and Paolo is right to be afraid"

# challenge is an integer modulo ec.n:
msghd2 = hf(msg2.encode()).digest()
c2 = int.from_bytes(msghd2, 'big') % ec.n
assert c2 != 0
print("c2:", hex(c2))

c2: 0x7adb91982ec03ef87efcae7f0199aefa231d8855e0bd03319460e58c0bd18049


## The Mistake 
Reuse the same ephemeral key as the previous message:

In [19]:
#very bad! Never reuse an ephemeral key!!!
k2 = k
print("k :", hex(k))
print("k2:", hex(k2))

k : 0x3fa3ccf6c168482533f1fa066650704546e56f0bf15fbfb3b4bc51f404e19ee7
k2: 0x3fa3ccf6c168482533f1fa066650704546e56f0bf15fbfb3b4bc51f404e19ee7


## Sign Second Message

In [20]:
K2 = mult(k2)

r = K2[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different k
assert r != 0

s2 = mod_inv(k2, ec.n) * (c2 + r*q) % ec.n
# if s2 == 0 (extremely unlikely for large ec.n) go back to a different k
assert s2 != 0

print("r :", hex(r))
print("s2:", hex(s2))

r : 0xf91a63e7574b8a7ea99cc8999456e8044b9d1cb05a3ec25c3e8886cee3ed0142
s2: 0xc7e70db59ce8c90521cc3d5599e8f7894422598673e16952b6da46b8f693afb8


## Verify Second Signature

In [18]:
w = mod_inv(s2, ec.n)
u = c2*w % ec.n
v = r*w % ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x % ec.n)

True


# Exercise
Because of the ephemeral key reuse is possible to calculate the private key from the 2 signatures. 

In [21]:
# forget k, k2, q
k=k2=q=0

# solve the problem of calculating q
# using only r, s, and s2:
# k = 
# q = 

print(hex(q))
print(mult(q) == Q) # check it is the correct private key

0x0
False
